## Analyzing a corpus of books using natural language processing (NLP)

### Books about Istanbul are scraped and is then further processed to get sentences based on the object list

In [2]:
import pprint
import nltk.corpus
import random
import os
import re
import gensim

In [3]:
import pandas as pd
import nltk
import string
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk import word_tokenize
from gensim.models import Word2Vec as w2v
from sklearn.decomposition import PCA

In [4]:
from gensim import corpora

In [5]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [6]:
folder_path = "/content/drive/MyDrive/Istanbul_Books"

In [7]:
def break_sentences(text):
    # replace newline characters with spaces
    text = re.sub(r'\n', ' ', text)

    # split the text into sentences based on .?! characters
    sentences = re.split(r'[.?!]+', text)

    # remove empty strings and leading/trailing spaces from sentences
    sentences = [s.strip() for s in sentences if s.strip()]

    return sentences

In [8]:
def clean_string(s):
    # remove any sequence of more than two spaces with a single space
    s = re.sub(r' {2,}', ' ', s)

    return s.strip()

In [ ]:
collection = []
read_books = []

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        # read the file contents
        with open(os.path.join(folder_path, filename), 'r', encoding="utf-8") as file:
            text = file.read()
        
        # break the text into sentences
        sentences = break_sentences(text)
        
        # add the sentences to the collection list
        collection += sentences
        
        # add the file name to the read_books list
        read_books.append(filename)
        
# print the names of the files that were read
print(f"Read {len(read_books)} files:")

# print the sentences
for i, sentence in enumerate(collection):
    print(f"Index {i}: {sentence}")

In [ ]:
final_collection = []

for sentence in collection:
    cleaned_sentence = clean_string(sentence)
    final_collection.append(cleaned_sentence)

for i, sentence in enumerate(final_collection):
    print(f"Index {i}: {sentence}")

In [11]:
len(final_collection)

239739

In [ ]:
stoplist = set("and the or but nor yet so been now will are were would should did dont ever with was had have has".split(' '))

texts = [[word.replace(".","").replace(",","").replace("'","").replace(":","") for word in document.lower().split()] 
         for document in final_collection]

texts = [[word for word in text if (word not in stoplist and len(word)>2)] 
         for text in texts]

to_delete = []
for i in range(len(texts)):
    t = texts[i]
    test = [w for w in t if w.isalpha()]
    if len(test) < 5:
        to_delete.append(i)
    else:
        texts[i] = test

for i in sorted(to_delete, reverse = True):
    del texts[i]
    del final_collection[i]
        
# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
pprint.pprint(processed_corpus)

In [13]:
len(processed_corpus)

166551

In [14]:
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary<43345 unique tokens: ['architecture', 'art', 'certain', 'city', 'culture']...>


In [ ]:
print(dictionary.token2id)

In [16]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

In [17]:
from gensim import models

tfidf = models.TfidfModel(bow_corpus)

In [18]:
from gensim.similarities import Similarity
from gensim.test.utils import get_tmpfile

In [19]:
index_tmpfile = get_tmpfile("index")

index = Similarity(index_tmpfile, bow_corpus, num_features = len(dictionary))

In [20]:
query_document = "balcony".lower().split()
query_bow = dictionary.doc2bow(query_document)

In [21]:
sims = index[query_bow]

In [ ]:
for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
    print(document_number, score)

In [23]:
top_scores = sorted(enumerate(sims), key=lambda x: x[1], reverse=True)[:10]

In [24]:
top_sentences = [final_collection[idx] for idx, score in top_scores]

# combine the top paragraphs and their corresponding books into a single string
output = ""
for i in range(len(top_sentences)):
    output += f"Sentence {i+1}: {top_sentences[i]}\n"
        
# write the output to a file named "top_paragraphs.txt"
with open("./balcony.txt", "w", encoding="utf-8") as f:
    f.write(output)

print(output)

Sentence 1: Nalan leaned over the balcony railing
Sentence 2: ‘You can’t stand on the balcony and throw things at strangers
Sentence 3: Women often use a screened-off area or a balcony
Sentence 4: The views from its upper-storey balcony are stunning
Sentence 5: “Come out here,” Joy calls from the balcony
Sentence 6: Macit went out onto the balcony
Sentence 7: No, they did not swing from the balcony
Sentence 8: No, they did not hang off the balcony
Sentence 9: Flat Number 7: Me Ants raided my balcony today – or perhaps it was just today that I noticed ants had raided my balcony
Sentence 10: The dried aubergines were hanging from the kitchen balcony

